# 카글 텍스트 분류 - 합성곱 신경망 활용 접근방법

In [3]:
import sys
import os
import numpy as np
import json

from sklearn.model_selection import train_test_split

import tensorflow as tf
from tensorflow.keras import layers
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing import sequence

/Users/sinseongjin/tf110/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: compiletime version 3.5 of module 'tensorflow.python.framework.fast_tensor_util' does not match runtime version 3.6
  return f(*args, **kwds)


In [20]:
DEFAULT_DIR ='~/.kaggle/competitions/word2vec-nlp-tutorial/'
DATA_IN_DIR = './data_in/'
DATA_OUT_DIR = './data_out/'
INPUT_TRAIN_DATA_FILE_NAME = 'train_input.npy'
LABEL_TRAIN_DATA_FILE_NAME = 'train_label.npy'
INPUT_TEST_DATA_FILE_NAME = 'test_input.npy'

DATA_CONFIGS_FILE_NAME = 'data_configs.json'

train_input_data = np.load(open(DATA_IN_DIR + INPUT_TRAIN_DATA_FILE_NAME, 'rb'))
train_label_data = np.load(open(DATA_IN_DIR + LABEL_TRAIN_DATA_FILE_NAME, 'rb'))
test_input_data = np.load(open(DATA_IN_DIR + INPUT_TEST_DATA_FILE_NAME, 'rb'))

prepro_configs = None

with open(DATA_IN_DIR + DATA_CONFIGS_FILE_NAME, 'r') as f:
    prepro_configs = json.load(f)

In [19]:
# 파라메터 변수
RNG_SEED = 1234
BATCH_SIZE = 128
NUM_EPOCHS = 1
VOCAB_SIZE = len(prepro_configs)
EMB_SIZE = 128
VALID_SPLIT = 0.2
MAX_SEQ_LEN = 604 # 문장 최대 길이

train_input, eval_input, train_label, eval_label = train_test_split(train_input_data, train_label_data, test_size=VALID_SPLIT, random_state=RNG_SEED)

#문장 길이 구하는 값, 전처리로 같은 길이를 맞추어 놔서 의미 없음
train_len = np.array([min(len(x), MAX_SEQ_LEN) for x in train_input])
eval_len = np.array([min(len(x), MAX_SEQ_LEN) for x in eval_input])

## tf.data 세팅

In [6]:
def mapping_fn(X, Y=None):
    input, label = {'x': X}, Y
    return input, label

def train_input_fn():
    dataset = tf.data.Dataset.from_tensor_slices((train_input, train_label))
    dataset = dataset.shuffle(buffer_size=len(train_input))
    dataset = dataset.batch(BATCH_SIZE)
    dataset = dataset.map(mapping_fn)
    dataset = dataset.repeat(count=NUM_EPOCHS)

    iterator = dataset.make_one_shot_iterator()
    
    return iterator.get_next()

def eval_input_fn():
    dataset = tf.data.Dataset.from_tensor_slices((eval_input, eval_label))
    dataset = dataset.shuffle(buffer_size=len(eval_input))
    dataset = dataset.batch(BATCH_SIZE)
    dataset = dataset.map(mapping_fn)

    iterator = dataset.make_one_shot_iterator()
    
    return iterator.get_next()

## 모델 세팅

In [7]:
def model_fn(features, labels, mode, params):

    TRAIN = mode == tf.estimator.ModeKeys.TRAIN
    EVAL = mode == tf.estimator.ModeKeys.EVAL
    PREDICT = mode == tf.estimator.ModeKeys.PREDICT
    
    #embedding layer를 선언합니다.
    input_layer = tf.contrib.layers.embed_sequence(
                    features['x'],
                    VOCAB_SIZE,
                    EMB_SIZE,
                    initializer=params['embedding_initializer']
                    )
    
    # 현재 모델이 학습모드인지 여부를 확인하는 변수입니다.
    training = (mode == tf.estimator.ModeKeys.TRAIN)    
    # embedding layer에 대한 output에 대해 dropout을 취합니다.
    dropout_emb = tf.layers.dropout(inputs=input_layer,
                                   rate=0.5,
                                   training=training)
    
    
    conv = tf.layers.conv1d(
            inputs=dropout_emb,
            filters=32,
            kernel_size=3,
            padding='same',
            activation=tf.nn.relu)
    
    pool = tf.reduce_max(input_tensor=conv, axis=1)
    hidden = tf.layers.dense(inputs=pool, units=250, activation=tf.nn.relu)
    dropout_hidden = tf.layers.dropout(inputs=hidden, rate=0.2, training=training)
    logits = tf.layers.dense(inputs=dropout_hidden, units=1, name='logits')
    
    #prediction 진행 시, None
    if labels is not None:
        labels = tf.reshape(labels, [-1, 1])

    if TRAIN:
        global_step = tf.train.get_global_step()
        loss = tf.losses.sigmoid_cross_entropy(labels, logits)
        train_op = tf.train.AdamOptimizer(0.001).minimize(loss, global_step)

        return tf.estimator.EstimatorSpec(mode=mode, train_op=train_op, loss = loss)
    
    elif EVAL:
        loss = tf.losses.sigmoid_cross_entropy(labels, logits)
        pred = tf.nn.sigmoid(logits)
        accuracy = tf.metrics.accuracy(labels, tf.round(pred))
        return tf.estimator.EstimatorSpec(mode=mode, loss=loss, eval_metric_ops={'acc': accuracy})
        
    elif PREDICT:
        return tf.estimator.EstimatorSpec(
            mode=mode,
            predictions={
                'prob': tf.nn.sigmoid(logits)
            }
        )

In [8]:
params = {'embedding_initializer': tf.random_uniform_initializer(-1.0, 1.0)}

model_dir = os.path.join(os.getcwd(), "data_out/checkpoint/cnn/")
os.makedirs(model_dir, exist_ok=True)

config_tf = tf.estimator.RunConfig()
config_tf._save_checkpoints_secs = 100
config_tf._keep_checkpoint_max =  2
config_tf._log_step_count_steps = 100

cnn_est = tf.estimator.Estimator(model_fn, model_dir=model_dir, config=config_tf, params=params)
cnn_est.train(train_input_fn) #학습하기
cnn_est.evaluate(eval_input_fn) #평가하기

INFO:tensorflow:Using config: {'_model_dir': '/Users/sinseongjin/github/DeepNLP/7.NLPBOOK/4.TEXT_CLASSIFICATION/data_out/checkpoint/cnn/', '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_steps': None, '_save_checkpoints_secs': 100, '_session_config': None, '_keep_checkpoint_max': 2, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': 100, '_train_distribute': None, '_device_fn': None, '_service': None, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x12a805278>, '_task_type': 'worker', '_task_id': 0, '_global_id_in_cluster': 0, '_master': '', '_evaluation_master': '', '_is_chief': True, '_num_ps_replicas': 0, '_num_worker_replicas': 1}
INFO:tensorflow:Calling model_fn.


KeyboardInterrupt: 

# 테스트 데이터셋

In [9]:
# 예측된 모델을 불러 체크포인트로 결과치를 불러온다.
test_input_data = np.load(open(DATA_IN_DIR + INPUT_TEST_DATA_FILE_NAME, 'rb')) #테스트데이터 로드

predict_input_fn = tf.estimator.inputs.numpy_input_fn(x={"x":test_input_data}, shuffle=False) #numpy 형태로 저장
cnn_est.predict(input_fn=predict_input_fn)

predictions = np.array([p['prob'][0] for p in cnn_est.predict(input_fn=predict_input_fn)])

INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from /Users/sinseongjin/github/DeepNLP/7.NLPBOOK/4.TEXT_CLASSIFICATION/data_out/checkpoint/cnn/model.ckpt-314
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.


In [10]:
import pandas as pd
#테스트 데이터 로드
test = pd.read_csv(DEFAULT_PATH+"testData.tsv", header=0, delimiter="\t", quoting=3 )

print ("test dataset shape: {}".format(test.shape))

#알아보기 쉽게 데이터랑 붙여두는 편이 좋을 거 같습니다.
output = pd.DataFrame( data={"id":test["id"], "sentiment":list(predictions)} )

#지금까지 처리한 결과를 파일로 저장합니다.
output.to_csv("./data_out/Bag_of_Words_model_test.csv", index=False, quoting=3 )

NameError: name 'DEFAULT_PATH' is not defined

# Serving (Test용도입니다)

In [11]:
LIMIT_SEQ_LEN = 604

# Serving을 제공하기 위한 입력 리시버 함수를 선언해주어야 한다.
def serving_input_receiver_fn():
    # estimator에 입력하고자 하는 데이터를 dict 객체로 정의한다.
    receiver_tensor = {
        # 외부로부터 입력을 받는 프로토콜은 스트링이다. ServingInputReceiver 메뉴얼에도 언급되었다시피
        # 이 방식은 TFRecord 파일 형태로 시리얼화한 데이터 형태로 전송을 받는다. (이를 tf.example 방식이라고도 하는 것 같다)
        'x': tf.placeholder(dtype=tf.string, shape=[None])
    }
    
    # 다음은 TFRecord 방식으로 받은 데이터를 모델에 넣을 수 있게 처리를 하는 dict 객체라 보면 된다.
    # 쉽게 말하면 앞서 estimator를 진행하기 위해 data_fn의 과정을 작성해두는데 이 과정을 여기서 거친다 보면 된다.
    features = {
        key: tensor
        for key, tensor in receiver_tensor.items()
    }
    # TFRecord로 시리얼화 된 데이터를 integer tensor로 변환하기 위해서는 string to int로 decode를 해줘야 한다.
    fn = lambda query: tf.decode_raw(query, tf.int64)
    features['x'] = tf.map_fn(fn, features['x'], dtype=tf.int64)
    # 받은 데이터에 대해 모델입력에 맞는 shape로 구성을 해주기 위해 reshape을 해준다.
    features['x'] = tf.reshape(features['x'], [-1, LIMIT_SEQ_LEN])

    # 위에 정의한 받을 데이터에 대한 프로토콜과 모델에 입력할 데이터 전처리를 다음 함수 파라메터에 입력해준다.
    return tf.estimator.export.ServingInputReceiver(features, receiver_tensor)

In [12]:
export_dir_base = DATA_OUT_DIR + './served_model/new_staging'

# 서빙에 대한 입력 리시버함수와 저장 위치를 파라메터로 지정한다면, 서빙 pb파일로 저장하여 간단하게 모델을 활용할 수 있게 된다.
# 실행을 하게 되면 저장된 파일의 위치를 텍스트 출력을 통해 얻게된다.
path = cnn_est.export_savedmodel(export_dir_base, serving_input_receiver_fn) #,
#                       strip_default_attrs=True)

INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Signatures INCLUDED in export for Classify: None
INFO:tensorflow:Signatures INCLUDED in export for Regress: None
INFO:tensorflow:Signatures INCLUDED in export for Predict: ['serving_default']
INFO:tensorflow:Signatures INCLUDED in export for Train: None
INFO:tensorflow:Signatures INCLUDED in export for Eval: None
INFO:tensorflow:Restoring parameters from /Users/sinseongjin/github/DeepNLP/7.NLPBOOK/4.TEXT_CLASSIFICATION/data_out/checkpoint/cnn/model.ckpt-314
INFO:tensorflow:Assets added to graph.
INFO:tensorflow:No assets to write.
INFO:tensorflow:SavedModel written to: ./data_out/./served_model/new_staging/temp-b'1540693100'/saved_model.pb


In [13]:
# 모델이 저장된 경로 위치를 파라메터로 지정하여 함수를 부르면 간단하게 예측 모델을 활용할 수 있다.
# 이 모델 예측을 간단하게 함수로 받게된다.
predictor_fn = tf.contrib.predictor.from_saved_model(
    export_dir = path,
    # 옵션이지만 실행되는 모델에 대한 이름을 명시하고자 한다면 다음의 파라메터를 활용한다.
    signature_def_key="serving_default"
)

# 입력 데이터는 array 형태로 입력을 할 수가 있다. 이 때 주의할 점은 데이터 타입을 
# 반드시 입력 리시버 함수에서 정의한 데이터 타입과 일치시켜야한다.
# 보통 int32, int64 데이터에 대한 정의가 맞지않아 막상 데이터를 모델에 입력했을 때 
# 바이너리 데이터 길이의 불일치 문제로 이어져 에러가 발생할 수 있다.

d = np.array(train_input[0], dtype=np.int64)
print('입력 데이터 array: ', d)

d = d.tostring()
print('\n모델에 입력하기 위한 array: ', d)

INFO:tensorflow:Restoring parameters from ./data_out/./served_model/new_staging/1540693100/variables/variables
입력 데이터 array:  [ 4353   728     1  2173    76   928    72  1941    97   208   123   209
 16272  1748  1806    55   200  2352   528   906    97   767    79    29
   175    97   696   147    90  4612   199    32  4353    93    24 53786
   220  1462 11785  6274     1     1   629   174    67     1   612   171
  2847   203     5    15    16    20 53787   191   279   136    96    24
  2220   213  2023   315  3695    16     4  1989   763    28     1  3928
   297   261    72    16  1046   203     9  4353   214    31   737   329
    54    98     0     0     0     0     0     0     0     0     0     0
     0     0     0     0     0     0     0     0     0     0     0     0
     0     0     0     0     0     0     0     0     0     0     0     0
     0     0     0     0     0     0     0     0     0     0     0     0
     0     0     0     0     0     0     0     0     0     0     0     

In [14]:
# 간단하게 입력 파라메터에 대한 프로토콜만 맞춰 입력한다면 예츨 모델을 함수를 통해 얻을 수 있다.
output = predictor_fn({'x': [d]})

In [15]:
output

{'prob': array([[0.30063093]], dtype=float32)}

In [28]:
def str2vec(s, vocab, limit_len=604, PAD='pad', UNK='unk'):
    s_tok = s.split()
    pad_count = limit_len - len(s_tok)
    indecied_list = [vocab[t] if t in vocab else 0 for t in s_tok]
    if pad_count > 0:
        indecied_list = indecied_list + ([0] * pad_count)
    elif pad_count < 0:
        indecied_list = indecied_list[:limit_len]
    
    np_index = np.array(indecied_list, dtype=np.int64)
    np_index = np_index.tostring()
    
    return indecied_list, np_index

In [29]:
vocab = prepro_configs

test_sent = 'i feel this movie is so good'
arr, input_arr = str2vec(test_sent, vocab)
print(arr)

[0, 122, 0, 1, 0, 0, 5, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0

In [30]:
predictor_fn({'x': [input_arr]})

{'prob': array([[0.60136044]], dtype=float32)}

In [32]:
input_str = input('input text : ')
arr, input_arr = str2vec(input_str, vocab)
predictor_fn({'x': [input_arr]})

input text : i think the movie is really bad


{'prob': array([[0.24812755]], dtype=float32)}